# Queries with and without Azure OpenAI

Now that we have our Search Engine loaded **from two different data sources in two diferent indexes**, we are going to try some example queries and then use Azure OpenAI service to see if we can get even better results.

The idea is that a user can ask a question about Computer Science (first datasource/index) or about Covid (second datasource/index), and the engine will respond accordingly.
This **Multi-Index** demo, mimics the scenario where a company loads multiple type of documents of different types and about completly different topics and the search engine must respond with the most relevant results.

## Set up variables

In [1]:
import os
import urllib
import requests
import random
from collections import OrderedDict
from IPython.display import display, HTML
from langchain.chains import LLMChain
from langchain.prompts import PromptTemplate
from langchain.llms import AzureOpenAI
from langchain.chat_models import AzureChatOpenAI
from langchain.vectorstores import FAISS
from langchain.docstore.document import Document
from langchain.chains.question_answering import load_qa_chain
from langchain.chains.qa_with_sources import load_qa_with_sources_chain
from langchain.embeddings import OpenAIEmbeddings
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from common.prompts import COMBINE_QUESTION_PROMPT, COMBINE_PROMPT
from common.utils import model_tokens_limit, num_tokens_from_docs

from dotenv import load_dotenv
load_dotenv("credentials.env")

# Demo Datasource Blob Storage. Change if using your own data
DATASOURCE_SAS_TOKEN = "?sv=2022-11-02&ss=bf&srt=sco&sp=rltfx&se=2023-11-29T01:50:59Z&st=2023-05-10T16:50:59Z&spr=https&sig=ZT7MLy%2BnlvAxUKKj5v0RwoWObXaab3gO4ec2%2Fci6iL0%3D"

In [2]:
# Setup the Payloads header
headers = {'Content-Type': 'application/json','api-key': os.environ['AZURE_SEARCH_KEY']}

## Multi-Index Search queries

In [3]:
# Index that we are going to query (from Notebook 01 and 02)
index1_name = "cogsrch-index-files"
index2_name = "cogsrch-index-csv"
indexes = [index1_name, index2_name]

Try questions that you think might be answered or addressed in computer science papers in 2020-2021 or that can be addressed by medical publications about COVID in 2020-201. Try comparing the results with the open version of ChatGPT.<br>
The idea is that the answers using Azure OpenAI only looks at the information contained on these publications.

**Example Questions you can ask**:
- What is CLP?
- How Markov chains work?
- What are some examples of reinforcement learning?
- What are the main risk factors for Covid-19?
- What medicine reduces inflamation in the lungs?
- Why Covid doesn't affect kids that much compared to adults?
- Does chloroquine really works against covid?
- tell me Use cases where I can use deep learning to solve it

In [4]:
QUESTION = "What is CLP?" 

### Search on both indexes individually and aggragate results

**Note**: In order to standarize the indexes we are setting 4 mandatory fields to be present on each index: id, title, content, pages, language. These fields must be present in each index so that each document can be treated the same along the code.

In [5]:
agg_search_results = []

for index in indexes:
    url = os.environ['AZURE_SEARCH_ENDPOINT'] + '/indexes/'+ index + '/docs'
    url += '?api-version={}'.format(os.environ['AZURE_SEARCH_API_VERSION'])
    url += '&search={}'.format(QUESTION)
    url += '&select=*'
    url += '&$top=10'  # You can change this to anything you need/want
    url += '&queryLanguage=en-us'
    url += '&queryType=semantic'
    url += '&semanticConfiguration=my-semantic-config'
    url += '&$count=true'
    url += '&speller=lexicon'
    url += '&answers=extractive|count-3'
    url += '&captions=extractive|highlight-false'

    resp = requests.get(url, headers=headers)
    print(url)
    print(resp.status_code)

    search_results = resp.json()
    agg_search_results.append(search_results)
    print("Results Found: {}, Results Returned: {}".format(search_results['@odata.count'], len(search_results['value'])))

https://cog-search-4njhjc5ogtyfm.search.windows.net/indexes/cogsrch-index-files/docs?api-version=2021-04-30-Preview&search=What is CLP?&select=*&$top=10&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-false
200
Results Found: 9789, Results Returned: 10
https://cog-search-4njhjc5ogtyfm.search.windows.net/indexes/cogsrch-index-csv/docs?api-version=2021-04-30-Preview&search=What is CLP?&select=*&$top=10&queryLanguage=en-us&queryType=semantic&semanticConfiguration=my-semantic-config&$count=true&speller=lexicon&answers=extractive|count-3&captions=extractive|highlight-false
200
Results Found: 22435, Results Returned: 10


### Display the top results (from both searches) based on the score

In [6]:
display(HTML('<h4>Top Answers</h4>'))

for search_results in agg_search_results:
    for result in search_results['@search.answers']:
        if result['score'] > 0.5: # Show answers that are at least 50% of the max possible score=1
            display(HTML('<h5>' + 'Answer - score: ' + str(round(result['score'],2)) + '</h5>'))
            display(HTML(result['text']))
            
print("\n\n")
display(HTML('<h4>Top Results</h4>'))

content = dict()
ordered_content = OrderedDict()


for search_results in agg_search_results:
    for result in search_results['value']:
        if result['@search.rerankerScore'] > 1: # Filter results that are at least 25% of the max possible score=4
            content[result['id']]={
                                    "title": result['title'],
                                    "chunks": result['pages'],
                                    "language": result['language'], 
                                    "caption": result['@search.captions'][0]['text'],
                                    "score": result['@search.rerankerScore'],
                                    "name": result['metadata_storage_name'], 
                                    "location": result['metadata_storage_path']                  
                                }
    
#After results have been filtered we will Sort and add them as an Ordered list\n",
for id in sorted(content, key= lambda x: content[x]["score"], reverse=True):
    ordered_content[id] = content[id]
    url = ordered_content[id]['location'] + DATASOURCE_SAS_TOKEN
    title = str(ordered_content[id]['title']) if (ordered_content[id]['title']) else ordered_content[id]['name']
    score = str(round(ordered_content[id]['score'],2))
    display(HTML('<h5><a href="'+ url + '">' + title + '</a> - score: '+ score + '</h5>'))
    display(HTML(ordered_content[id]['caption']))

## Comments on Query results

As seen above the semantic search feature of Azure Cognitive Search service is good. It gives us some answers and also the top results with the corresponding file and the paragraph where the answers is possible located.

Let's see if we can make this better with Azure OpenAI

# Using Azure OpenAI

To use OpenAI to get a better answer to our question, the thought process is: let's send the the documents of the search result to the GPT model and let it understand the document's content and provide a better response.

We will use a genius library call **LangChain** that wraps a lot of boiler plate code.
Langchain is one library that does a lot of the prompt engineering for us under the hood, for more information see [here](https://python.langchain.com/en/latest/index.html)

## A gentle intro to chaining LLMs and prompt engineering

Chains are what you get by connecting one or more large language models (LLMs) in a logical way. (Chains can be built of entities other than LLMs but for now, let’s stick with this definition for simplicity).

Azure OpenAI is a type of LLM (provider) that you can use but there are others like Cohere, Huggingface, etc.

Chains can be simple (i.e. Generic) or specialized (i.e. Utility).

* Generic — A single LLM is the simplest chain. It takes an input prompt and the name of the LLM and then uses the LLM for text generation (i.e. output for the prompt).

Here’s an example:

In [7]:
# Set the ENV variables that Langchain needs to connect to Azure OpenAI
os.environ["OPENAI_API_BASE"] = os.environ["AZURE_OPENAI_ENDPOINT"]
os.environ["OPENAI_API_KEY"] = os.environ["AZURE_OPENAI_API_KEY"]
os.environ["OPENAI_API_VERSION"] = os.environ["AZURE_OPENAI_API_VERSION"]
os.environ["OPENAI_API_TYPE"] = "azure"

In [8]:
# Create our LLM model
# Make sure you have the deployment named "gpt-35-turbo" for the model "gpt-35-turbo (0301)". 
# Use "gpt-4" if you have it available.
MODEL = "gpt-35-turbo" # options: gpt-35-turbo, gpt-4, gpt-4-32k
llm = AzureChatOpenAI(deployment_name=MODEL, temperature=0, 
                      max_tokens=500, 
                      streaming=True)

In [9]:
# Now we create a simple prompt template
prompt = PromptTemplate(
    input_variables=["question", "language"],
    template='Answer the following question: "{question}". Give your response in {language}',
)

print(prompt.format(question=QUESTION, language="French"))

Answer the following question: "What is CLP?". Give your response in French


In [10]:
# And finnaly we create our first generic chain
chain_chat = LLMChain(llm=llm, prompt=prompt)
chain_chat({"question": QUESTION, "language": "French"})

{'question': 'What is CLP?',
 'language': 'French',
 'text': "CLP signifie Classification, Étiquetage et Emballage des substances et des mélanges. Il s'agit d'un système de classification et d'étiquetage harmonisé au niveau européen pour les produits chimiques dangereux. Le CLP vise à protéger la santé humaine et l'environnement en fournissant des informations claires et précises sur les dangers des produits chimiques."}

Great!!, now you know how to create a simple prompt and use a chain in order to answer a general question using ChatGPT knowledge!. 

It is important to note that we rarely use generic chains as standalone chains. More often they are used as building blocks for Utility chains (as we will see next). Also important to notice is that we are NOT using our documents or the result of the Azure Search yet, just the knowledge of ChatGPT on the data it was trained on.

**The second type of Chains are Utility:**

* Utility — These are specialized chains, comprised of many LLMs to help solve a specific task. For example, LangChain supports some end-to-end chains (such as [QA_WITH_SOURCES](https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html) for QnA Doc retrieval, Summarization, etc) and some specific ones (such as GraphQnAChain for creating, querying, and saving graphs). 

We will look at one specific chain called **qa_with_sources** in this workshop for digging deeper and solve our use case of enhancing the results of Azure Cognitive Search.


But before dealing with the utility chain needed, we need to deal first with this problem: **the content of the search result files is or can be very lengthy, more than the allowed tokens allowed by the GPT Azure OpenAI models**. So what we need to do is: split in chunks, vectorize those chunks and do a vector semantic search to get the top chunks in order to provide the best and not too lenghy context to the LLM.

Notice that **the documents chunks are already done in Azure Search**. *ordered_content* dictionary (created a few cells above) contains the pages (chunks) of each document. So we don't really need to chunk them again, but we still need to make sure that we can be as fast as possible and that we are below the max allowed input token limits of our selected OpenAI model.

In [11]:
# Iterate over each of the results chunks and create a LangChain Document class to use further in the pipeline
docs = []
for key,value in ordered_content.items():
    for page in value["chunks"]:
        docs.append(Document(page_content=page, metadata={"source": value["location"]}))
        
print("Number of chunks:",len(docs))

Number of chunks: 148


We need now to calculate the number of tokens for all the chunks combined to decide what to do:
1) Should we embed to vectors and do cosine similarity because there is too much data to fit on the prompt as context?
2) What happens if the amount of chunks is too big? how do we keep the response time on-check?

In [12]:
# Calculate number of tokens of our docs
if(len(docs)>0):
    tokens_limit = model_tokens_limit(MODEL) # this is a custom function we created in common/utils.py
    num_tokens = num_tokens_from_docs(docs) # this is a custom function we created in common/utils.py
    print("Custom token limit for", MODEL, ":", tokens_limit)
    print("Combined docs tokens count:",num_tokens)
        
else:
    print("NO RESULTS FROM AZURE SEARCH")

Custom token limit for gpt-35-turbo : 3000
Combined docs tokens count: 184936


Now, depending of the amount of chunks/pages returned from the search result, which is very related to the size of the documents returned, we need to make some decisions to keep the speed of the response from Azure OpenAI at reasonable levels.

**The logic is**: if there is less than X chunks (of 5000 chars each) to vectorize, then we use OpenAI models in all of them, which currently don't offer batch processing (but it will soon), but if there is more than X chunks we have to trim the number of chunks to the first X, otherwise large documents can take minutes to vectorize.

**Note**, this is a temporary solution. Once Vector Capabilities in Azure Cognitive Search is in Public Preview, then we can vectorize once, store, then retrieve them again when needed.


In [13]:
%%time

chunks_limit = 100 # This is the limit of how many chunks are we willing to take in order to keep the response fairly quick

if num_tokens > tokens_limit:
    # Select the Embedder model
    embedder = OpenAIEmbeddings(deployment="text-embedding-ada-002", chunk_size=1) 
    print("Initial Number of chunks:",len(docs))
    if len(docs) > chunks_limit:
        docs = docs[:chunks_limit]
        print("Truncated Number of chunks:",len(docs))
    
    # Create our in-memory vector database index from the chunks given by Azure Search.
    # We are using FAISS. https://ai.facebook.com/tools/faiss/
    db = FAISS.from_documents(docs, embedder)
    top_docs = db.similarity_search(QUESTION, k=4)  # Return the top 4 documents
    
    # Now we need to recalculate the tokens count of the top results from similarity vector search
    # in order to select the chain type: stuff (all chunks in one prompt) or 
    # map_reduce (multiple calls to the LLM to summarize/reduce the chunks and then combine them)
    
    num_tokens = num_tokens_from_docs(top_docs)
    print("Token count after similarity search:", num_tokens)
    chain_type = "map_reduce" if num_tokens > tokens_limit else "stuff"
    
else:
    # if total tokens is less than our limit, we don't need to vectorize and do similarity search
    top_docs = docs
    chain_type = "stuff"
    
print("Chain Type selected:", chain_type)

Initial Number of chunks: 148
Truncated Number of chunks: 100
Token count after similarity search: 3836
Chain Type selected: map_reduce
CPU times: user 750 ms, sys: 36.3 ms, total: 786 ms
Wall time: 6.07 s


At this point we already have the top most similar chunks (in order of relevance) in **top_docs**

Now we need Azure OpenAI GPT model to understand these top chunks and provide us an answer to the question.

For this task, we need to come back to the Utility Chain: **qa_with_sources** that we mentioned before. See [HERE](https://python.langchain.com/en/latest/modules/chains/index_examples/qa_with_sources.html) for reference.

We created our own custom prompts so we can add translation to a specified language. But, for more information on the different types of prompts for this utility chain please see [HERE](https://github.com/hwchase17/langchain/tree/master/langchain/chains/question_answering)


In [14]:
if chain_type == "stuff":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       prompt=COMBINE_PROMPT)
elif chain_type == "map_reduce":
    chain = load_qa_with_sources_chain(llm, chain_type=chain_type, 
                                       question_prompt=COMBINE_QUESTION_PROMPT,
                                       combine_prompt=COMBINE_PROMPT,
                                       return_intermediate_steps=True)

In [15]:
# Uncomment the below line if you want to check our custom COMBINE_PROMPT
# print(chain.combine_document_chain.llm_chain.prompt.template)

In [16]:
%%time
# Try with other language as well
response = chain({"input_documents": top_docs, "question": QUESTION, "language": "English"})

CPU times: user 71.8 ms, sys: 1.2 ms, total: 73 ms
Wall time: 20.3 s


In [17]:
answer = response['output_text']

display(HTML('<h4>Azure OpenAI ChatGPT Answer:</h4>'))
display(HTML(answer.split("SOURCES:")[0]))

sources_list = answer.split("SOURCES:")[1].replace(" ","").split(",")

sources_html = '<u>Sources</u>: '
for index, value in enumerate(sources_list):
    url = value + DATASOURCE_SAS_TOKEN
    sources_html +='<sup><a href="'+ url + '">[' + str(index+1) + ']</a></sup>'
    
display(HTML(sources_html))

In [18]:
# Uncomment if you want to inspect the results from map_reduce chain type, each top similar chunk summary (k=4 by default)

# if chain_type == "map_reduce":
#     for step in response['intermediate_steps']:
#         display(HTML("<b>Chunk Summary:</b> " + step))

# Summary
##### This answer is way better than taking just the result from Azure Cognitive Search. So the summary is:
- Azure Cognitive Search give us the top results (context)
- Azure OpenAI takes these results and understand the content and uses it as context to give the best answer
- Best of two worlds!

# NEXT
We just added a smart layer on top of Azure Cognitive Search. This is the backend for a GPT Smart Search Engine.

However, we are missing something: **How to have a conversation with this engine?**

On the next Notebook, we are going to understand the concept of **memory**. This is necessary in order to have a chatbot that can establish a conversation with the user. Without memory, there is no real conversation.